In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.color_palette("tab10")
from scipy import stats
from sklearn.model_selection import train_test_split
import os
from langdetect import detect
seed = 1349
import unicodedata
import re
import nltk
from bs4 import BeautifulSoup
import requests
from nltk.corpus import stopwords
import wrangle_g as wr

In [2]:
df = wr.prepare_mvp()

In [3]:
X_train, X_test, y_train, y_test = wr.final_wrangle(df)
X_train, X_test, y_train, y_test

(253    build statushttpstravisciorgppaciyasesvgbranch...
 193    machine learn az step towards data science mac...
 241    stats different graph tool create datasets who...
 278    pypi versionhttpsbadgefuryiopydduisvghttpsbadg...
 256    knip module aim make cod hassle free usage kni...
                              ...                        
 283    lookalike model b2b customers author rajesh gu...
 220    gretel client p alignleft img width15 srchttps...
 217    data machine learn bore way tutorial walk set ...
 333    cookie pythonhttpsimgshieldsiopypipyversionsco...
 336    stock prediction use facebook prophet amaze co...
 Name: stopped, Length: 232, dtype: object,
 203    p aligncenter br hrefhttpsimageflaticoncomicon...
 252    catch tweet keyword project allow extract data...
 281    datasciencesalaryestimator best lightmode fork...
 42     animate map repo show easily access one severa...
 32     common function data science project contain s...
 273    herokupythonmlexampl